# Projet 7 - Réalisez une analyse de sentiments grâce au Deep Learning

> 🎓 OpenClassrooms • Parcours [AI Engineer](https://openclassrooms.com/fr/paths/795-ai-engineer) | 👋 *Étudiant* : [David Scanu](https://www.linkedin.com/in/davidscanu14/)

## 📝 Contexte

Dans le cadre de ma formation d'AI Engineer chez OpenClassrooms, ce projet s'inscrit dans un scénario professionnel où j'interviens en tant qu'ingénieur IA chez MIC (Marketing Intelligence Consulting), entreprise de conseil spécialisée en marketing digital.

Notre client, Air Paradis (compagnie aérienne), souhaite **anticiper les bad buzz sur les réseaux sociaux**. La mission consiste à développer un produit IA permettant de **prédire le sentiment associé à un tweet**, afin d'améliorer la gestion de sa réputation en ligne.

## ⚡ Mission

> Développer un modèle d'IA permettant de prédire le sentiment associé à un tweet.

Créer un prototype fonctionnel d'un modèle d'**analyse de sentiments pour tweets** selon trois approches différentes :

1. **Modèle sur mesure simple** : Approche classique (régression logistique) pour une prédiction rapide
2. **Modèle sur mesure avancé** : Utilisation de réseaux de neurones profonds avec différents word embeddings
3. **Modèle avancé BERT** : Exploration de l'apport en performance d'un modèle BERT

Cette mission implique également la mise en œuvre d'une démarche MLOps complète :
- Utilisation de MLFlow pour le tracking des expérimentations et le stockage des modèles
- Création d'un pipeline de déploiement continu (Git + Github + plateforme Cloud)
- Intégration de tests unitaires automatisés
- Mise en place d'un suivi de performance en production via Azure Application Insight

## 🗓️ Plan de travail

1. **Exploration et préparation des données**
   - Acquisition des données de tweets Open Source
   - Analyse exploratoire et prétraitement des textes

2. **Développement des modèles**
   - Implémentation du modèle classique (régression logistique)
   - Conception du modèle avancé avec différents word embeddings
   - Test du modèle BERT pour l'analyse de sentiments
   - Comparaison des performances via MLFlow

3. **Mise en place de la démarche MLOps**
   - Configuration de MLFlow pour le tracking des expérimentations
   - Création du dépôt Git avec structure de projet appropriée
   - Implémentation des tests unitaires automatisés
   - Configuration du pipeline de déploiement continu

4. **Déploiement et monitoring**
   - Développement de l'API de prédiction avec FastAPI
   - Déploiement sur Heroku
   - Création de l'interface de test (Streamlit ou Next.js)
   - Configuration du suivi via Azure Application Insight

5. **Communication**
   - Rédaction de l'article de blog
   - Préparation du support de présentation

## Importation des bibliothèques

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import zipfile
import os
import re
import datetime
import warnings
import string

# Télécharger les ressources NLTK nécessaires
import nltk
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
from collections import Counter
from wordcloud import WordCloud
from nltk.stem import WordNetLemmatizer

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.family'] = 'DejaVu Sans'
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt_tab to /home/david/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /home/david/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/david/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## 💾 Jeu de données : Sentiment140

Le jeu de données [Sentiment140 dataset with 1.6 million tweets](https://www.kaggle.com/datasets/kazanova/sentiment140) est une ressource majeure pour l'analyse de sentiment sur Twitter, comprenant **1,6 million de tweets** extraits via l'API Twitter. Ces tweets ont été automatiquement annotés selon leur polarité sentimentale, offrant une base solide pour développer des modèles de classification de sentiment.

Le jeu de données est organisé en 6 colonnes distinctes :

1. **target** : La polarité du sentiment exprimé dans le tweet.
   - 0 = sentiment négatif
   - 2 = sentiment neutre
   - 4 = sentiment positif
2. **ids** : L'identifiant unique du tweet (exemple : *2087*)
3. **date** : La date et l'heure de publication du tweet.
4. **flag** : La requête utilisée pour obtenir le tweet.
   - Exemple : *lyx*
   - Si aucune requête n'a été utilisée : *NO_QUERY*
5. **user** : Le nom d'utilisateur de l'auteur du tweet.
6. **text** : Le contenu textuel du tweet.

In [2]:
%%time 

# Define the URL and the local file path
url = "https://s3-eu-west-1.amazonaws.com/static.oc-static.com/prod/courses/files/AI+Engineer/Project+7%C2%A0-+D%C3%A9tectez+les+Bad+Buzz+gr%C3%A2ce+au+Deep+Learning/sentiment140.zip"
local_zip_path = "./content/data/sentiment140.zip"
extract_path = "./content/data"

if not os.path.exists(extract_path):

    # Create the directory if it doesn't exist
    os.makedirs(extract_path, exist_ok=True)

    # Download the zip file
    response = requests.get(url)
    with open(local_zip_path, 'wb') as file:
        file.write(response.content)

    # Extract the contents of the zip file
    with zipfile.ZipFile(local_zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)

    # Delete the zip file
    os.remove(local_zip_path)

CPU times: user 712 μs, sys: 37 μs, total: 749 μs
Wall time: 754 μs


In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)

In [4]:

# Define the path to the CSV file
csv_file_path = os.path.join(extract_path, 'training.1600000.processed.noemoticon.csv')

# Define the column names
column_names = ['target', 'ids', 'date', 'flag', 'user', 'text']

# Load the dataset into a pandas DataFrame
raw_data = pd.read_csv(csv_file_path, encoding='latin-1', names=column_names)

# Display the first few rows of the DataFrame
raw_data.head()

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [5]:
print(f"Ce dataframe contient {raw_data.shape[0]} lignes et {raw_data.shape[1]} colonnes.")

Ce dataframe contient 1600000 lignes et 6 colonnes.


### Recommandations de modélisation

Approches recommandées pour l'analyse de sentiment:

1. Approches classiques de Machine Learning:
   - Modèles basés sur les sacs de mots (BoW) ou TF-IDF avec classifieurs comme Régression Logistique, SVM, Random Forest ou Naive Bayes
   - Avantages: rapides à entraîner, interprétables
   - Inconvénients: ne capturent pas la sémantique et l'ordre des mots

2. Word Embeddings + Classifieurs:
   - Utiliser des embeddings pré-entraînés (Word2Vec, GloVe, FastText) avec des classifieurs ML
   - Avantages: capture la sémantique des mots, gère les mots hors vocabulaire (FastText)
   - Inconvénients: perd l'information de séquence

3. Réseaux de neurones récurrents (RNN, **LSTM**, GRU):
   - Avantages: capture l'information séquentielle, efficace pour le texte
   - Inconvénients: temps d'entraînement plus long, risque de surapprentissage

4. Modèles transformers (BERT, Sentence Transformers, RoBERTa, DistilBERT):
   - Fine-tuning de modèles pré-entraînés spécifiques à Twitter comme BERTweet
   - Avantages: état de l'art en NLP, capture le contexte bidirectionnel
   - Inconvénients: coûteux en ressources, complexe à déployer

5. Approches d'ensemble:
   - Combiner plusieurs modèles pour obtenir de meilleures performances
   - Avantages: souvent meilleures performances, plus robuste
   - Inconvénients: complexité accrue, temps d'inférence plus long

Considérations importantes:

1. Déséquilibre des classes: utiliser des techniques comme SMOTE, sous-échantillonnage, ou pondération des classes
2. Validation croisée: essentielle pour évaluer correctement les performances
3. Métriques d'évaluation: ne pas se limiter à l'accuracy, utiliser F1-score, précision, rappel, et AUC-ROC
4. Interprétabilité: pour certaines applications, privilégier des modèles interprétables ou utiliser SHAP/LIME
5. Dépendance temporelle: considérer l'évolution du langage sur Twitter au fil du temps

**Notes du mentor :**

Voici le texte extrait de cette capture d'écran :

- Création de deux modèles de Deep Learning, dont au moins un avec un layer LSTM.
- Simulation selon deux techniques de pré-traitement (lemmatization, stemming) sur l'un des 2 modèles, afin de choisir la technique pour la suite des simulations.
- Simulation selon 2 approches de word embedding (parmi Word2VEc, Glove, FastText), entraînés avec le jeu de données ou pré-entraînés sur au moins un des 2 modèles de Deep Learning, afin de choisir l'embedding pour la suite des simulations.
- Création ensuite d'un modèle BERT, il y a 2 approches possibles :
  - Générer des features (sentence embedding) à partir d'un TFBertModel (Hugging Face) ou d'un d'un model via le Hub TensorFlow, puis ajouter une ou des couches de classification
  - Utiliser directement un modèle Hugging Face de type TFBertForSequenceClassification
- En option tester USE (Universal Sentence Encoding) pour le feature engineering

Problèmes et erreurs courants :
- Temps de traitement et limitation de ressources en TensorFlow-Keras.
- Inspirer des exemples de modèles cités en ressources


## Approches classiques de Machine Learning

In [32]:
df = raw_data.copy()

In [33]:
def convert_sentiment_label(df):
    df['target'] = df['target'].apply(lambda x: 0 if x == 0 else 1)
    return df

df = convert_sentiment_label(df)

In [34]:
df['target'].value_counts()

target
0    800000
1    800000
Name: count, dtype: int64

In [37]:
from multiprocessing import Pool

# Fonction de prétraitement pour les tweets
def preprocess_tweet(tweet):
    """
    Prétraite un tweet en appliquant plusieurs transformations :
    - Conversion en minuscules
    - Remplacement des URLs, mentions et hashtags par des tokens spéciaux
    - Suppression des caractères spéciaux
    - Tokenisation et lemmatisation
    - Suppression des stopwords
    """
    # Vérifier si le tweet est une chaîne de caractères
    if not isinstance(tweet, str):
        return ""
    
    # Convertir en minuscules
    tweet = tweet.lower()
    
    # Remplacer les URLs par un token spécial
    tweet = re.sub(r'https?://\S+|www\.\S+', '<URL>', tweet)
    
    # Remplacer les mentions par un token spécial
    tweet = re.sub(r'@\w+', '<MENTION>', tweet)
    
    # Traiter les hashtags (conserver le # comme token séparé et le mot qui suit)
    tweet = re.sub(r'#(\w+)', r'# \1', tweet)
    
    # Supprimer les caractères spéciaux et les nombres, mais garder les tokens spéciaux
    tweet = re.sub(r'[^\w\s<>@#!?]', '', tweet)
    
    # Tokenisation
    tokens = word_tokenize(tweet)
    
    # Lemmatisation
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    # Supprimer les stopwords, mais conserver les négations importantes
    stop_words = set(stopwords.words('english'))
    important_words = {'no', 'not', 'nor', 'neither', 'never', 'nobody', 'none', 'nothing', 'nowhere'}
    stop_words = stop_words - important_words
    tokens = [token for token in tokens if token not in stop_words]
    
    # Rejoindre les tokens en une chaîne
    return ' '.join(tokens)


def process_in_parallel(df, func, n_jobs=4):
    """
    Applique une fonction à un DataFrame en le divisant en parties et en traitant chaque partie en parallèle.
    Permet d'accélérer le traitement sur les ordinateurs multi-coeurs.
    """
    df_split = np.array_split(df, n_jobs)
    pool = Pool(n_jobs)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

def apply_preprocessing(df_part):
    df_part['processed_text'] = df_part['text'].apply(preprocess_tweet)
    return df_part

In [38]:
# Préparation des données pour l'entraînement

def downsample_data(df, n_samples=50000):
    """
    Réduit la taille d'un DataFrame en échantillonnant aléatoirement un nombre spécifié de lignes pour chaque classe.
    """
    negative_samples = df[df['target'] == 0].sample(n=n_samples, random_state=42)
    positive_samples = df[df['target'] == 1].sample(n=n_samples, random_state=42)
    return pd.concat([negative_samples, positive_samples])

balanced_df = downsample_data(df, n_samples=50000)
balanced_df['target'].value_counts()

target
0    50000
1    50000
Name: count, dtype: int64

In [39]:
# Appliquer le prétraitement à tous les tweets
print("Prétraitement des tweets en cours...")
# balanced_df['processed_text'] = balanced_df['text'].apply(preprocess_tweet)
# Utilisation
balanced_df = process_in_parallel(balanced_df, apply_preprocessing, n_jobs=8)
print("Prétraitement terminé !")

Prétraitement des tweets en cours...
Prétraitement terminé !


In [40]:
balanced_df.head()

,target,ids,date,flag,user,text,processed_text
212188,0,1974671194,Sat May 30 13:36:31 PDT 2009,NO_QUERY,simba98,@xnausikaax oh no! where did u order from? tha...,< MENTION > oh no ! u order ? thats horrible
299036,0,1997882236,Mon Jun 01 17:37:11 PDT 2009,NO_QUERY,Seve76,A great hard training weekend is over. a coup...,great hard training weekend couple day rest le...
475978,0,2177756662,Mon Jun 15 06:39:05 PDT 2009,NO_QUERY,x__claireyy__x,"Right, off to work Only 5 hours to go until I...",right work 5 hour go im free xd
588988,0,2216838047,Wed Jun 17 20:02:12 PDT 2009,NO_QUERY,Balasi,I am craving for japanese food,craving japanese food
138859,0,1880666283,Fri May 22 02:03:31 PDT 2009,NO_QUERY,djrickdawson,Jean Michel Jarre concert tomorrow gotta work...,jean michel jarre concert tomorrow got ta work...


In [41]:
# Diviser les données en ensembles d'entraînement et de test
from sklearn.model_selection import train_test_split

X = balanced_df['processed_text']
y = balanced_df['target']

# Division train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Taille de l'ensemble d'entraînement: {X_train.shape[0]} exemples")
print(f"Taille de l'ensemble de test: {X_test.shape[0]} exemples")

Taille de l'ensemble d'entraînement: 80000 exemples
Taille de l'ensemble de test: 20000 exemples


In [ ]:
# Vectorisation des textes avec TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

# Créer les vectoriseurs
tfidf_vectorizer = TfidfVectorizer(max_features=10000, ngram_range=(1, 2))
bow_vectorizer = CountVectorizer(max_features=10000, ngram_range=(1, 2))

# Transformer les textes en vecteurs TF-IDF
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Transformer les textes en vecteurs BoW
X_train_bow = bow_vectorizer.fit_transform(X_train)
X_test_bow = bow_vectorizer.transform(X_test)

print("Vectorisation terminée !")

Taille de l'ensemble d'entraînement: 80000 exemples
Taille de l'ensemble de test: 20000 exemples
Vectorisation terminée !
